In [1]:
# 모듈 import
import sys
import os

# 현재 노트북 파일의 절대 경로
current_path = os.path.abspath('')

# 상위 폴더 경로
parent_path = os.path.dirname(current_path)

# 상위 폴더를 sys.path에 추가
sys.path.append(parent_path)

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [18]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
#print(cfg.data.train)
#print(cfg.model)
#cfg = Config.fromfile('/data/ephemeral/home/Jihwan/level2-objectdetection-cv-07/mmdetection/ssd_experiment/ssd300_coco.py')
cfg = Config.fromfile('/data/ephemeral/home/Jihwan/level2-objectdetection-cv-07/mmdetection/configs/ssd/ssd300_coco.py')



root='/data/ephemeral/home/dataset/'

# dataset config 수정
cfg.data.train.dataset.classes = classes
cfg.data.train.dataset.img_prefix = root
cfg.data.train.dataset.ann_file = root + 'train.json' # train json 정보
cfg.data.train.dataset.pipeline[4]['img_scale'] = (512,512) # Resize ssd
cfg.data.train.times = 1
cfg.data.img_norm_cfg = dict(mean=[123.675, 116.28, 110.53], std=[60, 59, 61], to_rgb=True)
cfg.data.type = 'ClassBalancedDataset'
cfg.data.oversample_thr = 0.1
#cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize faster-rcnn

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/ssd300_coco_trash_1'

cfg.model.bbox_head.num_classes = 10
#cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [19]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [20]:
# dataset 확인
datasets[0]

In [21]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-10-11 20:57:33,650 - mmcv - INFO - initialize SSDVGG with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://vgg16_caffe'}
2024-10-11 20:57:33,651 - mmcv - INFO - load model from: open-mmlab://vgg16_caffe
2024-10-11 20:57:33,652 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://vgg16_caffe
2024-10-11 20:57:33,724 - mmcv - INFO - initialize SSDNeck with init_cfg [{'type': 'Xavier', 'distribution': 'uniform', 'layer': 'Conv2d'}, {'type': 'Constant', 'val': 1, 'layer': 'BatchNorm2d'}, {'type': 'Constant', 'val': 20, 'override': {'name': 'l2_norm'}}]
2024-10-11 20:57:33,745 - mmcv - INFO - initialize SSDHead with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform', 'bias': 0}
2024-10-11 20:57:33,762 - mmcv - INFO - 
backbone.features.0.weight - torch.Size([64, 3, 3, 3]): 
PretrainedInit: load from open-mmlab://vgg16_caffe 
 
2024-10-11 20:57:33,763 - mmcv - INFO - 
backbone.features.0.bias - torch.Size([64]): 
PretrainedInit: load from ope

In [22]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-10-11 20:57:34,032 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-10-11 20:57:34,034 - mmdet - INFO - Start running, host: root@instance-12171, work_dir: /data/ephemeral/home/Jihwan/level2-objectdetection-cv-07/mmdetection/ssd_experiment/work_dirs/ssd300_coco_trash_2
2024-10-11 20:57:34,034 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_N

2024-10-11 20:57:44,321 - mmdet - INFO - Epoch [1][50/1221]	lr: 1.978e-04, eta: 1:40:14, time: 0.206, data_time: 0.055, memory: 2616, loss_cls: 15.3974, loss_bbox: 3.1695, loss: 18.5670, grad_norm: 70.5635
2024-10-11 20:57:52,109 - mmdet - INFO - Epoch [1][100/1221]	lr: 3.976e-04, eta: 1:27:57, time: 0.156, data_time: 0.007, memory: 2616, loss_cls: 8.8614, loss_bbox: 3.0967, loss: 11.9581, grad_norm: 27.7950
2024-10-11 20:57:59,839 - mmdet - INFO - Epoch [1][150/1221]	lr: 5.974e-04, eta: 1:23:34, time: 0.155, data_time: 0.006, memory: 2616, loss_cls: 5.7874, loss_bbox: 2.9605, loss: 8.7480, grad_norm: 51.3649
2024-10-11 20:58:07,634 - mmdet - INFO - Epoch [1][200/1221]	lr: 7.972e-04, eta: 1:21:28, time: 0.156, data_time: 0.007, memory: 2616, loss_cls: 5.3228, loss_bbox: 3.0043, loss: 8.3270, grad_norm: 39.8183
2024-10-11 20:58:15,344 - mmdet - INFO - Epoch [1][250/1221]	lr: 9.970e-04, eta: 1:20:00, time: 0.154, data_time: 0.007, memory: 2616, loss_cls: 5.3396, loss_bbox: 3.1546, loss: 